In [1]:
import sys

sys.path.insert(0, '..')

from app.processing.audio.SIWhisperModel import SIWhisperModel
from app.SIWeaviateClient import SIWeaviateClient
from app.processing.YoutubeProcessor import YoutubeProcessor

whisper = SIWhisperModel('medium', 'whisper-medium')



/home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/../app/schemas.py:44: GenericBeforeBaseModelWarning: Classes should inherit from `BaseModel` before generic classes (e.g. `typing.Generic[T]`) for pydantic generics to work properly.
  class ChunkWithScore(Generic[ChunkType], BaseDocumentChunk):
[nltk_data] Downloading package omw-1.4 to /home/erwan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/erwan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
with SIWeaviateClient() as client:
    YoutubeProcessor(client=client, whisper=whisper, youtube_url="https://www.youtube.com/watch?v=sMsgjQEXcTA")

AUDIO CODEC aac


whisper-medium: 100%|██████████| 183.066/183.066 [00:10<00:00, 16.90s/s]          


time_taken 11.42659854888916


: 